# DCA in backtest
Here to more simulate the trading, we use backtest module and implement a DCA strategy to verify it.
The goodness of `class Market` is we only need to care about how to react with it.  This allow user simulate more complicate trading operation like
sending order, cancel order,...etc

In [8]:
# Run it when crypto-backtest is updated, and then restart the kernel
!pip uninstall crypto-backtest -y
!pip install git+https://github.com/BullionBear/crypto-backtest.git

Found existing installation: crypto-backtest 0.0.9
Uninstalling crypto-backtest-0.0.9:
  Successfully uninstalled crypto-backtest-0.0.9
  Cloning https://github.com/BullionBear/crypto-backtest.git to /tmp/pip-req-build-3qhvg2hk
  Running command git clone --filter=blob:none --quiet https://github.com/BullionBear/crypto-backtest.git /tmp/pip-req-build-3qhvg2hk
  Resolved https://github.com/BullionBear/crypto-backtest.git to commit a111f944dd0f5adf83cab52276362892a30399bd
  Preparing metadata (setup.py) ... done
  Created wheel for crypto-backtest: filename=crypto_backtest-0.0.10-py3-none-any.whl size=9450 sha256=44bc86ca748db9914d1c05560cdba76b7badf15f1f02c1f1ad0a85693e75637f
  Stored in directory: /tmp/pip-ephem-wheel-cache-er3ldwfd/wheels/14/d1/5f/193347d44b51670dd287000ac535cdb7a8fc3f84e426532382
Successfully built crypto-backtest

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import backtest
backtest.__version__

'develop_0.0.10+'

In [2]:
from backtest.components.core import KLineIteratorFromFileSystem
from backtest.components import KLineMarket

from datetime import datetime, timezone

start_time = datetime(2023, 3, 8, tzinfo=timezone.utc).timestamp() * 1000
end_time = datetime(2023, 4, 8, tzinfo=timezone.utc).timestamp() * 1000
data_source = '/home/yite/crypto_data/binance/data'
kline_iterator = KLineIteratorFromFileSystem('BTCUSDT', start_time, end_time, data_source)
market = KLineMarket(kline_iterator)

In [3]:
market.get_ts()

1678237199999

In [4]:
market.next()

True

In [9]:
from backtest.models import MarketOrder

total_usdt = 1_000_000
invest_usdt = 30_000

while True:
    klines = []
    while market.is_kline():
        klines.append(market.get_kline())
    for kline in klines:
        dt = datetime.fromtimestamp(kline.close_time / 1000)
        price = kline.close
        amount = price / invest_usdt
        if dt.hour == 0:
            market_order = MarketOrder(
                timestamp=kline.close_time, 
                base='BTC', 
                quote='USDT', 
                side='BUY', 
                amount=amount
            )
            market.send_market(market_order)
    if not market.next():
        break
    

In [11]:
while market.is_trade():
    trade = market.get_trade()
    print(trade)

In [5]:
market.get_ts()

1678240799999

In [8]:
market_order = MarketOrder(
    timestamp=1640995200,  # Example timestamp
    base="BTC",
    quote="USD",
    side="buy",
    amount=1.0
    # order_id is optional since it defaults to uuid4
)